In [90]:
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, RemoveMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from CONFIG import GROQ_MODEL, TEMPERATURE
from langgraph.checkpoint.memory import InMemorySaver

In [91]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [92]:
class state_graph(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    summary: str

In [93]:
def chat_node(state: state_graph):
    messages = []

    if state['summary']:
        messages.append(
            SystemMessage(content=f"conversation summary:\n {state['summary']}")
        )
    messages.extend(state['messages'])

    response = llm.invoke(messages)
    return {'messages': [response]}

In [94]:
def summarization_node(state: state_graph):
    existing_summary = state['summary']

    if existing_summary:
        prompt = (
            f"""
        Existing summary is \n {existing_summary} \n
        Extand the summary using the new conversation above"""
      )
        
    else:
        prompt = "Generate the summary of above conversation"
        
    messages_for_sumamry = state['messages'] + [HumanMessage(content=prompt)]
    response = llm.invoke(messages_for_sumamry)

    messages_for_delete = state['messages'][:-2]
    return {
        'summary': response.content,
        'messages': [RemoveMessage(id=m.id) for m in messages_for_delete]
    }

In [95]:
def should_summarize(state: state_graph):
    return len(state['messages']) > 6

In [96]:
builder = StateGraph(state_graph)

builder.add_node('chat_node', chat_node)
builder.add_node('summarization_node', summarization_node)

builder.add_edge(START, 'chat_node')
builder.add_conditional_edges(
    'chat_node',
    should_summarize,
    {
        True: 'summarization_node',
        False: END
    }
)

builder.add_edge('summarization_node', END)

In [97]:
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [98]:
config = {'configurable': {'thread_id': 'thread-1'}}
output = graph.invoke({'messages': [HumanMessage(content="agentic ai just inshort")], 'summary':''}, config=config)
print(f"Human Question: {output['messages'][0].content}")
print(f"AI Response: {output['messages'][-1].content}")

Human Question: agentic ai just inshort
AI Response: Agentic AI refers to artificial intelligence systems that can act independently, making decisions and taking actions based on their own goals and motivations, rather than just following pre-programmed instructions.


In [99]:
config = {'configurable': {'thread_id': 'thread-1'}}
output = graph.invoke({'messages': [HumanMessage(content="any banifit of n8n")], 'summary':''}, config=config)
print(f"Human Question: {output['messages'][0].content}")
print(f"AI Response: {output['messages'][-1].content}")

Human Question: agentic ai just inshort
AI Response: n8n is a workflow automation tool that offers several benefits, including:

1. **Low-code automation**: n8n allows users to automate workflows without extensive coding knowledge.
2. **Customizable**: Highly customizable, enabling users to create tailored workflows for specific needs.
3. **Integration with various services**: Supports integration with multiple services, such as APIs, databases, and messaging platforms.
4. **Flexible workflow design**: Enables users to design complex workflows using a visual interface.
5. **Error handling and debugging**: Provides features for error handling and debugging, making it easier to troubleshoot workflows.
6. **Scalability**: Designed to handle large volumes of data and scale with growing needs.
7. **Open-source**: n8n is open-source, which means it's free to use and modify.
8. **Community support**: Has an active community of users and developers who contribute to its growth and provide supp

In [100]:
out = graph.get_state(config).values['messages']
out

[HumanMessage(content='agentic ai just inshort', additional_kwargs={}, response_metadata={}, id='ce0b1efc-5370-4c28-89dd-596a406b92b8'),
 AIMessage(content='Agentic AI refers to artificial intelligence systems that can act independently, making decisions and taking actions based on their own goals and motivations, rather than just following pre-programmed instructions.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 41, 'total_tokens': 77, 'completion_time': 0.102995779, 'completion_tokens_details': None, 'prompt_time': 0.001947081, 'prompt_tokens_details': None, 'queue_time': 0.008472987, 'total_time': 0.10494286}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb855-4b97-78f1-9d92-e26ffb312f61-0', usage_metadata={'input_tokens': 41, 'output_tokens': 36, 'total_tokens': 77}),
 HumanMessag

In [101]:
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='agentic ai just inshort', additional_kwargs={}, response_metadata={}, id='ce0b1efc-5370-4c28-89dd-596a406b92b8'), AIMessage(content='Agentic AI refers to artificial intelligence systems that can act independently, making decisions and taking actions based on their own goals and motivations, rather than just following pre-programmed instructions.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 41, 'total_tokens': 77, 'completion_time': 0.102995779, 'completion_tokens_details': None, 'prompt_time': 0.001947081, 'prompt_tokens_details': None, 'queue_time': 0.008472987, 'total_time': 0.10494286}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb855-4b97-78f1-9d92-e26ffb312f61-0', usage_metadata={'input_tokens': 41, 'output_tokens': 36, '